# MEET THE NET
### Pattern matching

In [23]:
using System.Runtime.CompilerServices;

enum Operation { SystemTest, Start, Stop, Reset }
class State 
{ 
    public static State CreateState([CallerMemberName]string message = "") 
    { 
        Console.WriteLine(message);
        return new State();
    }
}

State PerformOperation(Operation command) =>
   command switch
   {
       Operation.SystemTest => RunDiagnostics(),
       Operation.Start => StartSystem(),
       Operation.Stop => StopSystem(),
       Operation.Reset => ResetToReady(),
       _ => throw new ArgumentException("Invalid enum value for command", nameof(command)),
   };

State PerformOperation(string command) =>
   command switch
   {
       "SystemTest" => RunDiagnostics(),
       "Start" => StartSystem(),
       "Stop" => StopSystem(),
       "Reset" => ResetToReady(),
       _ => throw new ArgumentException("Invalid string value for command", nameof(command)),
   };

State PerformOperation(string command, int seconds) =>
   command switch
   {
       "SystemTest" when seconds < 10 => RunDiagnostics(),
       "Start" when seconds >= 10 && seconds <= 20 => StartSystem(),
       "Stop" => StopSystem(),
       "Reset" => ResetToReady(),
       _ => throw new ArgumentException("Invalid string value for command", nameof(command)),
   };

State RunDiagnostics() => State.CreateState();
State StartSystem() => State.CreateState();
State StopSystem() => State.CreateState();
State ResetToReady() => State.CreateState();

PerformOperation("Start");

StartSystem


In [24]:
string WaterState(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        (> 32) and (< 212) => "liquid",
        < 32 => "solid",
        > 212 => "gas",
        32 => "solid/liquid transition",
        212 => "liquid / gas transition",
    };

string WaterState2(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        < 32 => "solid",
        32 => "solid/liquid transition",
        < 212 => "liquid",
        212 => "liquid / gas transition",
        _ => "gas",
};

Console.WriteLine(WaterState(10));
Console.WriteLine(WaterState2(10));

solid
solid


In [25]:
class Order
{
    public int Items { get; set; }
    public decimal Cost { get; set; }

    public void Deconstruct(out int items, out decimal cost)
    {
        items = Items;
        cost = Cost;
    }
}

decimal CalculateDiscount(Order order) =>
    order switch
    {
        { Items: > 10, Cost: > 1000.00m } => 0.10m,
        { Items: > 5, Cost: > 500.00m } => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => someObject.Cost,
    };

decimal CalculateDiscount2(Order order) =>
    order switch
    {
        ( > 10,  > 1000.00m) => 0.10m,
        ( > 5, > 50.00m) => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => someObject.Cost,
    };

Console.WriteLine(CalculateDiscount(new Order { Items = 10, Cost = 20m }));
Console.WriteLine(CalculateDiscount2(new Order { Items = 10, Cost = 20m }));

20
20


In [26]:
using System.Globalization;

List<string[]> stringList = new List<string[]>()
{
    new string[] { "04-01-2020", "DEPOSIT"   , "Initial deposit",                     "2250.00" },
    new string[] { "04-15-2020", "DEPOSIT"   , "Refund"         ,                     "125.65"  },
    new string[] { "04-18-2020", "DEPOSIT"   , "Paycheck"       ,                     "825.65"  },
    new string[] { "04-22-2020", "WITHDRAWAL", "Debit"          , "Groceries",        "255.73"  },
    new string[] { "05-01-2020", "WITHDRAWAL", "#1102"          , "Rent"     , "apt", "2100.00" },
    new string[] { "05-02-2020", "INTEREST"  ,                                        "0.65"    },
    new string[] { "05-07-2020", "WITHDRAWAL", "Debit"          , "Movies"   ,        "12.57"   },
    new string[] { "04-15-2020", "FEE"       ,                                        "5.55"    },
};

decimal balance = 0m;
foreach (string[] transaction in stringList)
{
    balance += transaction switch
    {
        [_, "DEPOSIT"   , _ , var amount] => decimal.Parse(amount, CultureInfo.InvariantCulture),
        [_, "WITHDRAWAL", .., var amount] => -decimal.Parse(amount, CultureInfo.InvariantCulture),
        [_, "INTEREST"  ,     var amount] => decimal.Parse(amount, CultureInfo.InvariantCulture),
        [_, "FEE"       ,     var fee]    => -decimal.Parse(fee, CultureInfo.InvariantCulture),
        _                                 => throw new InvalidOperationException($"Record {string.Join(", ", transaction)} is not in the expected format!"),
    };
    Console.WriteLine($"Record: {string.Join(", ", transaction)}, New balance: {balance:C}");
}

Record: 04-01-2020, DEPOSIT, Initial deposit, 2250.00, New balance: 2 250,00 zł
Record: 04-15-2020, DEPOSIT, Refund, 125.65, New balance: 2 375,65 zł
Record: 04-18-2020, DEPOSIT, Paycheck, 825.65, New balance: 3 201,30 zł
Record: 04-22-2020, WITHDRAWAL, Debit, Groceries, 255.73, New balance: 2 945,57 zł
Record: 05-01-2020, WITHDRAWAL, #1102, Rent, apt, 2100.00, New balance: 845,57 zł
Record: 05-02-2020, INTEREST, 0.65, New balance: 846,22 zł
Record: 05-07-2020, WITHDRAWAL, Debit, Movies, 12.57, New balance: 833,65 zł
Record: 04-15-2020, FEE, 5.55, New balance: 828,10 zł


In [27]:
public static bool IsLetter(this char c) =>
    c is >= 'a' and <= 'z' or >= 'A' and <= 'Z';

public static bool IsLetterOrSeparator(this char c) =>
    c is (>= 'a' and <= 'z') or (>= 'A' and <= 'Z') or '.' or ',';

### Ref locals

In [28]:
class NumberStore
{
    int[] numbers = { 1, 3, 7, 15, 31, 63, 127, 255, 511, 1023 };

    public ref int FindNumber(int target)
    {
        for (int ctr = 0; ctr < numbers.Length; ctr++)
        {
            if (numbers[ctr] >= target)
                return ref numbers[ctr];
        }
        return ref numbers[0];
    }

    public override string ToString() => string.Join(" ", numbers);
}

static void NumberStoreTest()
{
    var store = new NumberStore();
    Console.WriteLine($"Original sequence: {store.ToString()}");
    int number = 16;
    ref var value = ref store.FindNumber(number);
    value *= 2;
    Console.WriteLine($"New sequence:      {store.ToString()}");
}

NumberStoreTest();

Original sequence: 1 3 7 15 31 63 127 255 511 1023
New sequence:      1 3 7 15 62 63 127 255 511 1023


## Deconstructors

In [ ]:
class Sideable<T>
{
    public T Right { get; set; }
    public T Left { get; set; }

    public void Deconstruct(out T left, out T right)
    {
        left = Left;
        right = Right;
    }
}

var serialNumbers = new Sideable<string>
{
    Right = "123",
    Left  = "321"
};

var (leftSn, rightSn) = serialNumbers;
Console.WriteLine($"{leftSn}, {rightSn}");

In [29]:
public static void Deconstruct(this DateTime dateTime, out int year, out int month, out int day)
{
    year = dateTime.Year;
    month = dateTime.Month;
    day = dateTime.Day;
}

var dateTime = new DateTime(2023, 02, 16);
var (y, m, d) = dateTime;
Console.WriteLine($"{y}, {m}, {d}");

2023, 2, 16


### Readonly struct

In [ ]:
readonly struct Point
{
    public readonly double X;
    public readonly double Y;

    public double Z { get; }
}

### Readonly instance members

In [41]:
class Point
{
    public double X;
    public double Y;

    public readonly double DoSomeStuff()
    {
        Foo();
        return X + Y;
    }

    public readonly double Foo()
    {
        //X = X * 2;
        return 3.0;
    }
}

Error: (11,28): error CS0106: The modifier 'readonly' is not valid for this item
(17,28): error CS0106: The modifier 'readonly' is not valid for this item

## Nullable reference types (Nullable context)

In [ ]:
#nullable enable

class Bottle 
{ 
    public double Price { get; set; }

    public string Label { get; set; }
}

Bottle? b = new Bottle { Label = null };

In [42]:
#nullable enable

class MultiPack
{
    public List<Bottle>? Bottles { get; set; }

}

double PriceOfMultiPack(MultiPack? multiPack)
{
    double sum = 0.0;
    foreach(var bottle in multiPack!.Bottles!)
    {
        sum += bottle.Price;
    }

    return sum;
}


### Init only setters

In [ ]:
public struct WeatherObservation
{
    // we can add required keyword
    public required DateTime RecordedAt { get; init; }
    public decimal TemperatureInCelsius { get; init; }
    public decimal PressureInMillibars { get; init; }

    public override string ToString() =>
        $"At {RecordedAt:h:mm tt} on {RecordedAt:M/d/yyyy}: " +
        $"Temp = {TemperatureInCelsius}, with {PressureInMillibars} pressure";
}

var now = new WeatherObservation 
{ 
    RecordedAt = DateTime.Now, 
    TemperatureInCelsius = 20, 
    PressureInMillibars = 998.0m 
};

//now.TemperatureInCelsius = 18;

### Record types

#### Kotlin example
```kotlin
data class User(val name: String, val age: Int)
```

In [44]:
public record SmallPerson(string FirstName, string LastName);

public record BigPerson
{
    public string FirstName { get; init; } = default!;
    public string LastName { get; init; } = default!;
}

var person = new SmallPerson("MT", "WP");
var (first, last) = person;
(string f, _) = person;
Console.WriteLine($"{first}, {last}");

MT, WP


In [43]:
var person2 = person with { LastName = "ZP" };
person2

FirstName,LastName
MT,ZP


In [ ]:
public abstract record Person(string FirstName, string LastName);

public record Teacher(string FirstName, string LastName, int Grade)
    : Person(FirstName, LastName);
    
public record Student(string FirstName, string LastName, int Grade)
    : Person(FirstName, LastName);

In [45]:
public readonly record struct Point(double X, double Y, double Z);

public readonly struct StructPoint
{
    public double X { get; }
    public double Y { get; }
    public double Z { get; }

    public StructPoint(double x, double y, double z)
    {
        X = x; Y = y; Z = z;
    }
}

var point = new Point(1, 2, 3);
//point.X = 3;

Error: (16,1): error CS8852: Init-only property or indexer 'Point.X' can only be assigned in an object initializer, or on 'this' or 'base' in an instance constructor or an 'init' accessor.

## Fit and finish features

In [46]:
List<WeatherObservation> observations = new();
var observations2 = new List<WeatherObservation>();

public class WeatherForecast
{
    public WeatherForecast(DateTime dateTime, List<WeatherObservation> observations)
    {

    }
}

var forecast = new WeatherForecast(new(2023, 2, 16), new());

## Global using directives

## File-scoped namespace declaration

## Generic attributes

In [47]:
public class TypeAttribute : Attribute
{
    public TypeAttribute(Type type) { }
}

public class GenericAttribute<T> : Attribute { }

[TypeAttribute(typeof(string))]
public string NotGenMethod() => default;

[GenericAttribute<string>()]
public string GenMethod() => default;

public class GenericType<T>
{
   [GenericAttribute<int>()] // Not allowed! generic attributes must be fully constructed types.
   public string Method() => default;
}

Error: (16,5): error CS8968: 'T': an attribute type argument cannot use type parameters

## Raw string literals

In [48]:
string longMessage = """
    This is a long message.
    It has several lines.
        Some are indented 
                more than others.
    Some should start at the first column.
    Some have "quoted text" in them.
    """;

Console.WriteLine(longMessage);

This is a long message.
It has several lines.
    Some are indented
            more than others.
Some should start at the first column.
Some have "quoted text" in them.


# Interfaces

### Default implementations

In [ ]:
interface ICoordinateSystem<T>
{

    public int AxisCount { get; }

    public T GetAt(int axis);
    protected void SetAt(int axis, T value);

    public IEnumerable<T> AsEnumerable()
    {
        return Enumerable.Range(0, AxisCount).Select(i => GetAt(i));
    }

    public static ICoordinateSystem<T> Create2D(T x, T y)
        => new CoordinateSystemBase<T>(x, y);
}

class CoordinateSystemBase<T> : ICoordinateSystem<T>
{

    private T[] axes;

    public int AxisCount => axes.Length;

    public CoordinateSystemBase(params T[] axes)
    {
        this.axes = new T[axes.Length];
        Array.Copy(axes, this.axes, axes.Length);
    }

    public T GetAt(int axis) => this.axes[axis];
    public void SetAt(int axis, T value) => this.axes[axis] = value;

}

CoordinateSystemBase<double> coordSysBase = new CoordinateSystemBase<double>(10.0, 15.0);
coordSysBase.SetAt(0, 0.0);

ICoordinateSystem<double> coordSys = coordSysBase;
//coordSys.SetAt(0, 0.0);

ICoordinateSystem<double> coordSys2D = ICoordinateSystem<double>.Create2D(10, 13);
coordSysBase.SetAt(0, 0.0);

In [ ]:
interface ICoordinateSystem2D<T> : ICoordinateSystem<T>
{
    public T X 
    {
        get => GetAt(0);
        set => SetAt(0, value);    
    }

    public T Y
    {
        get => GetAt(1);
        set => SetAt(1, value);
    }
}

In [ ]:
interface IDataProcessor
{
    public void ProcessData(string path);

    public abstract static string[] SupportedFormats();
}

class MusicProcessor : IDataProcessor
{
    public void ProcessData(string path)
    {
        // Do stuff
    }

    public static string[] SupportedFormats()
        => new string[] { "mp3", "wav" };
}

class VideoProcessor : IDataProcessor
{

    public void ProcessData(string path)
    {
        // Do stuff
    }

    public static string[] SupportedFormats()
        => new string[] { "mp4", "avi" };

}

void ProcessData<T>(T processor) where T : IDataProcessor
{
    if (T.SupportedFormats().Contains("mp4"))
    {
        return;
    }
}

### Generic operators

In [22]:
using System.Numerics;

interface IPoint<T, TSelf> where TSelf : IPoint<T, TSelf>
{
    public T X { get; }
    public T Y { get; }

    public abstract static TSelf operator *(TSelf left, T right);
}

class DoublePoint : IPoint<double, DoublePoint>
{   
    public double X { get; }
    public double Y { get; }

    public DoublePoint(double x, double y) 
        => (X, Y) = (x, y);

    public static DoublePoint operator *(DoublePoint left, double right)
    {
        return new DoublePoint(left.X * right, left.Y * right);
    }
}

https://learn.microsoft.com/en-us/dotnet/standard/generics/math

In [ ]:
using System.Numerics;
int x = 0;

IMinMaxValue<int> minMax = x;
